### EDA (Exploratory Data Analysis) & Data Cleaning

In [ ]:

##import das bibliotecas e adequando colunas, linhas e formato de números

from google.cloud import bigquery
from dotenv import load_dotenv
import pandas as pd
import pandas_gbq as gbq
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re

# Carrega o .env
load_dotenv()

#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"

# Verificar se a configuração está correta
print("Credenciais do BigQuery:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

# Verifica se a variável está configurada
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))




In [ ]:
# Configurar o cliente do BigQuery
client = bigquery.Client()

# Nome do dataset e tabela
dataset_id = 'raw_data'

In [ ]:
# Configurar Pandas para exibir todas as colunas e todas as linhas completas
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Listar tabelas no dataset
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)




In [ ]:
query = f"SELECT * FROM `raw_data.humanresources_employee`"
data = client.query(query).result().to_dataframe()

# Expandir a coluna JSON
raw_data_humanresources_employee_bkp = pd.json_normalize(data['data'])

#cópia do original
EDA_humanresources_employee = raw_data_humanresources_employee_bkp.copy()  


# Exibir os dados expandidos
print(EDA_humanresources_employee.head(n=10))



In [ ]:
#dimensões do df antes de remover duplicatas

EDA_humanresources_employee.shape

In [ ]:
# Identificar duplicatas com base em 'businessentityid'
duplicatas = EDA_humanresources_employee[EDA_humanresources_employee.duplicated(subset=['businessentityid'], keep=False)]

# Verificar se existem duplicatas
if not duplicatas.empty:
    # Ordenar duplicatas por 'businessentityid' e 'modifieddate'
    duplicatas_ordenadas = duplicatas[duplicatas.duplicated(subset=['businessentityid'], keep=False)].sort_values(by=['businessentityid'])

    # Exibir duplicatas ordenadas
    print("Duplicatas ordenadas:")
    print(duplicatas_ordenadas)
else:
    print("Não foram encontradas duplicatas.")


In [ ]:
# Remover duplicatas mantendo a última ocorrência com base em 'modifieddate'
EDA_humanresources_employee = EDA_humanresources_employee.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicatas (baseando-se na última 'modifieddate'): {len(EDA_humanresources_employee)}")

#cópia dados brutos
raw_data_bkp_2_sem_duplicatas = EDA_humanresources_employee.copy()


In [ ]:
# Ordenar o DataFrame por 'businessentityid' e 'modifieddate'
EDA_humanresources_employee = EDA_humanresources_employee.sort_values(by=['businessentityid', 'modifieddate'])

print(EDA_humanresources_employee)




In [ ]:
#Certifique-se de que as colunas de datas está sendo reconhecida corretamente como contendo valores nulos (NaN em pandas). (Não pode object)

print(EDA_humanresources_employee.info())


In [ ]:

# Identificar as colunas de data
date_columns = ['birthdate', 'hiredate', 'modifieddate']

# Converter todas as colunas para datetime
for col in date_columns:
    EDA_humanresources_employee[col] = pd.to_datetime(
        EDA_humanresources_employee[col], errors='coerce'
    )

# Criar uma cópia do DataFrame para exportação no formato JSON
datas_formatadas = EDA_humanresources_employee.copy()

# Formatar colunas no formato ISO 8601 para BigQuery e tratar nulos como null
for col in date_columns:
    datas_formatadas[col] = datas_formatadas[col].apply(
        lambda x: x.isoformat() if not pd.isnull(x) else None
    )

print(EDA_humanresources_employee.info())



In [ ]:
# Iterar por todas as colunas do DataFrame

for column in EDA_humanresources_employee.columns:
    # Verificar valores ausentes na coluna
    missing_rows = EDA_humanresources_employee[EDA_humanresources_employee[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
    # Mostrar as primeiras linhas ausentes (limitar para não poluir a saída)
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")



In [ ]:
# Preencher 'modifieddate' ausente ou igual a 'hiredate', pois pode ser a ultima data de modificação no sistema.
EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'].isnull() | (EDA_humanresources_employee['modifieddate'] == pd.Timestamp('1900-01-01')), 'modifieddate'] = EDA_humanresources_employee['hiredate']

# Exibir as linhas ajustadas
print("Linhas onde 'modifieddate' foi ajustado para 'hiredate':")
print(EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'] == EDA_humanresources_employee['hiredate']])


In [ ]:
# Criar uma cópia do DataFrame para exportação no formato JSON
ajustes_date_time = EDA_humanresources_employee.copy()

In [251]:
# valores únicos por coluna

valores_unicos = EDA_humanresources_employee.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

Valores únicos incluindo NaN:
birthdate           275
businessentityid    290
currentflag           1
gender                2
hiredate            164
jobtitle             67
loginid             290
maritalstatus         2
modifieddate          4
nationalidnumber    290
organizationnode    290
rowguid             290
salariedflag          2
sickleavehours       51
vacationhours       100
hire_year             8
dtype: int64


In [ ]:
# dropar colunas vazias

In [253]:
EDA_humanresources_employee.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, 2591 to 845
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   birthdate         290 non-null    datetime64[ns]
 1   businessentityid  290 non-null    int64         
 2   currentflag       290 non-null    bool          
 3   gender            290 non-null    object        
 4   hiredate          290 non-null    datetime64[ns]
 5   jobtitle          290 non-null    object        
 6   loginid           290 non-null    object        
 7   maritalstatus     290 non-null    object        
 8   modifieddate      290 non-null    datetime64[ns]
 9   nationalidnumber  290 non-null    object        
 10  organizationnode  290 non-null    object        
 11  rowguid           290 non-null    object        
 12  salariedflag      290 non-null    bool          
 13  sickleavehours    290 non-null    int64         
 14  vacationhours     290 non-nu

In [ ]:
# Padronizar textos em title ou upper
EDA_humanresources_employee['jobtitle'] = EDA_humanresources_employee['jobtitle'].str.strip().str.title()
EDA_humanresources_employee['gender'] = EDA_humanresources_employee['gender'].str.strip().str.upper()
EDA_humanresources_employee['maritalstatus'] = EDA_humanresources_employee['maritalstatus'].str.strip().str.upper()


# Verificar valores únicos para garantir a padronização
print("Valores únicos em 'jobtitle':", EDA_humanresources_employee['jobtitle'].unique())
print("Valores únicos em 'gender':", EDA_humanresources_employee['gender'].unique())
print("Valores únicos em 'gender':", EDA_humanresources_employee['maritalstatus'].unique())


In [239]:
# Identificar colunas numéricas para análise 
numeric_columns = ['sickleavehours', 'vacationhours']

# Exibir estatísticas descritivas
print(EDA_humanresources_employee[numeric_columns].describe())

# Calcular limites para outliers (IQR - Intervalo Interquartil)
for col in numeric_columns:
    q1 = EDA_humanresources_employee[col].quantile(0.25)
    q3 = EDA_humanresources_employee[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Filtrar outliers
    outliers = EDA_humanresources_employee[(EDA_humanresources_employee[col] < lower_bound) | (EDA_humanresources_employee[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


       sickleavehours  vacationhours
count          290.00         290.00
mean            45.31          50.61
std             14.54          28.79
min             20.00           0.00
25%             33.00          26.25
50%             46.00          51.00
75%             58.00          75.00
max             80.00          99.00

Coluna: sickleavehours
Limite inferior: -4.5, Limite superior: 95.5
Outliers detectados (0):
Empty DataFrame
Columns: [sickleavehours]
Index: []

Coluna: vacationhours
Limite inferior: -46.875, Limite superior: 148.125
Outliers detectados (0):
Empty DataFrame
Columns: [vacationhours]
Index: []


In [244]:
# Definir regex para validar números (exemplo: apenas dígitos, 9 caracteres)
# acrescentei para ver se tinha um padrão, mas não tem
regex = r'^\d{9}$'

# Verificar valores inválidos
invalid_nationalid = EDA_humanresources_employee[~EDA_humanresources_employee['nationalidnumber'].astype(str).str.match(regex)]
print(f"Valores inválidos em 'nationalidnumber':\n{invalid_nationalid['nationalidnumber']}")


Valores inválidos em 'nationalidnumber':
89      42487730
116     56920285
989     24756624
1070    52541318
1124    95958330
1178    72636981
512     14417807
593      6298838
890     66073987
1673    33237992
2276     9659517
287     10708100
242     92096924
719      8066363
791     63179277
809     36151748
1601    58791499
1628     1662732
1736     7201901
2528    90888098
2555    82638150
2510    54759846
1727     1300049
998     45615666
233     63761469
206     25011600
71      56772045
278     97728960
1898    65848458
1763    60114406
800     87268837
1241    19312190
2159    20244403
2312    28414965
1466       30845
1583    60517918
1421    20269531
1934    58317344
557     61161660
980     90836195
Name: nationalidnumber, dtype: object


In [ ]:
# Criar um backup do DataFrame tratado
EDA_humanresources_employee_bkp_v2 = EDA_humanresources_employee.copy()

# Verificar o tamanho do backup e as primeiras linhas
print(f"Backup criado com {len(EDA_humanresources_employee_bkp_v2)} linhas.")
print(EDA_humanresources_employee_bkp_v2.head())


In [ ]:
# Verificar e documentar colunas existentes
print("Colunas mantidas no dataset:", EDA_humanresources_employee.columns.tolist())


In [246]:
# Listar colunas binárias esperadas
binary_columns = ['currentflag', 'salariedflag']

# Verificar valores únicos em colunas binárias
for col in binary_columns:
    unique_values = EDA_humanresources_employee[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}") 



Valores únicos em 'currentflag': [ True]
Valores únicos em 'salariedflag': [ True False]


In [247]:
# Contar valores em 'currentflag' e 'salariedflag'
print("Distribuição de 'currentflag':")
print(EDA_humanresources_employee['currentflag'].value_counts())

print("\nDistribuição de 'salariedflag':")
print(EDA_humanresources_employee['salariedflag'].value_counts())


#se vale a pena deletar ou não a coluna currentflag, já que só tem 1 valor e é true ?!

Distribuição de 'currentflag':
currentflag
True    290
Name: count, dtype: int64

Distribuição de 'salariedflag':
salariedflag
False    238
True      52
Name: count, dtype: int64


In [249]:
# 1. Verificar se todos os funcionários ativos têm currentflag = True, pois deveria ser false = demitido/desligado
print("Funcionários ativos errados:", EDA_humanresources_employee[EDA_humanresources_employee['currentflag'] != True])

# 2. Validar datas
print("Contratações futuras:", EDA_humanresources_employee[EDA_humanresources_employee['hiredate'] > pd.Timestamp.now()])
print("Modifieddate antes de hiredate:", EDA_humanresources_employee[EDA_humanresources_employee['modifieddate'] < EDA_humanresources_employee['hiredate']])





Funcionários ativos errados: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours, hire_year]
Index: []
Contratações futuras: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours, hire_year]
Index: []
Modifieddate antes de hiredate: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours, hire_year]
Index: []


birthdate           275
businessentityid    290
currentflag           1
gender                2
hiredate            164
jobtitle             67
loginid             290
maritalstatus         2
modifieddate          4
nationalidnumber    290
organizationnode    290
rowguid             290
salariedflag          2
sickleavehours       51
vacationhours       100
hire_year             8
dtype: int64

In [ ]:
# #Definir Dataset e Nome da Tabela
# dataset_id = 'raw_data_cleaned'  # Substitua pelo nome do seu Dataset
# table_id = 'humanresources_employee'   # Substitua pelo nome da sua Tabela
# full_table_id = f"{client.project}.{dataset_id}.{table_id}"  # Nome completo da tabela

# #Exportar para o BigQuery
# EDA_humanresources_employee.to_gbq(full_table_id, if_exists='replace', credentials=client._credentials)



## ESTATÍSTICA DESCRITIVA

In [ ]:
# Selecionar colunas relevantes para análise descritiva
cols_para_analise = ['sickleavehours', 'vacationhours', 'salariedflag']

# Garantir que as datas estejam no formato correto
EDA_humanresources_employee['hire_year'] = pd.to_datetime(EDA_humanresources_employee['hiredate']).dt.year

# Adicionar a nova coluna à lista
cols_para_analise.append('hire_year')

# Gerar estatísticas descritivas
analise_descritiva = EDA_humanresources_employee[cols_para_analise].describe(include='all')

# Substituir NaN por '-'
analise_descritiva = analise_descritiva.fillna('-')

print(analise_descritiva)


In [ ]:
#EDA_humanresources_employee.boxplot(column='vacationhours')


import matplotlib.pyplot as plt

# Criar boxplot usando matplotlib
plt.boxplot(EDA_humanresources_employee['vacationhours'])
plt.title("Boxplot de Vacation Hours")
plt.ylabel("Vacation Hours")
plt.show()
